



<h2 align="center">
    Project: Intermediate SQL Queries - #2
</h2>




**Project Description:**

To work with the New York City Airbnb dataset in this project, we need to create and connect to an SQLite database as seen in the steps within this notebook.



**Dataset**
- You can find the New York City Airbnb dataset on Kaggle: [New York City Airbnb Dataset](https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data)
- Download the dataset from the provided link and save it as 'AB_NYC_2019.csv' on your computer.
- On colab, click on the "Upload" button and select the 'AB_NYC_2019.csv' file from your local computer.

## Introduction
In this project, we will work with the New York City Airbnb dataset. Appliying intermediate data analysis tasks.

## Table of Contents
1. [Libraries and Database Setup](#database-setup)
2. [Loading the Dataset](#loading-dataset)
3. [SQL Queries](#sql-queries)
4. [Closing Connection](#closing-connection)
5. [Summary of Intermediate SQL Commands](#summary-commands)

# Section 1: Libraries and Database Setup <a name="database-setup"></a>

###  1.1: Import Libraries
Let's start by importing the necessary libraries.


In [1]:
!pip install kaggle
! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json

In [2]:
!pip install -q kaggle

In [3]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!kaggle datasets download -d dgomonov/new-york-city-airbnb-open-data

Dataset URL: https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data
License(s): CC0-1.0
 82% 2.00M/2.44M [00:00<00:00, 3.67MB/s]
100% 2.44M/2.44M [00:00<00:00, 3.80MB/s]


In [8]:
!unzip /content/new-york-city-airbnb-open-data.zip

Archive:  /content/new-york-city-airbnb-open-data.zip
  inflating: AB_NYC_2019.csv         
  inflating: New_York_City_.png      


In [5]:
import pandas as pd
import numpy as np
import sqlite3

###  1.2: Connect to the Database

Next, establish a connection to the SQLite database named 'airbnb.db'.
Use the sqlite3 library to create a connection object and store it in a variable called 'conn'.


In [6]:


# Connect to the SQLite database
conn = sqlite3.connect('airbnb.db')
cursor = conn.cursor()

# Section 2: Loading the Dataset <a name="loading-dataset"></a>

###  2.1: Load the Dataset
- We work with the 'listings' table from the New York City Airbnb dataset.
- Load the dataset from the CSV file 'AB_NYC_2019.csv' into a DataFrame named 'df'.
- Then, import the DataFrame into the 'listings' table in the database using the to_sql() method.

\\

Use the read_csv() method from pandas to read the CSV file.


In [9]:


# Load the dataset into the SQLite database
data_path = '/content/AB_NYC_2019.csv'
df = pd.read_csv(data_path)
df.to_sql('listings', conn, if_exists='replace', index=False)

48895

# Section 3: Intermediate SQL Tasks <a name="sql-queries"></a>

Query the database using SQL queries to analyze the data in the SQLite database. You can use the sqlite3 library to execute the queries and fetch the results.

###  3.1: Using Subquery for Aggregation
- Find listings with a price greater than or equal to the maximum price of entire home/apartment listings.

In [12]:
df['room_type'].unique()

array(['Private room', 'Entire home/apt', 'Shared room'], dtype=object)

In [15]:
#   the query by adding the SELECT, WHERE, and LIMIT statement

query = """  SELECT id, name, neighbourhood_group, price from listings


 WHERE price >= (select max(price) from listings where room_type='Entire home/apt')


LIMIT 5;
"""

In [16]:


result = pd.read_sql_query(query, conn)
print("\n - Using Subquery for Aggregation:")
print(result)


Example 2 - Using Subquery for Aggregation:
         id                                             name  \
0   7003697              Furnished room in Astoria apartment   
1  13894339  Luxury 1 bedroom apt. -stunning Manhattan views   
2  22436899                              1-BR Lincoln Center   

  neighbourhood_group  price  
0              Queens  10000  
1            Brooklyn  10000  
2           Manhattan  10000  


###  3.2: Using CTE for Clarity
- Create a [Common Table Expression (CTE)](https://learnsql.com/blog/what-is-common-table-expression/) for high-priced listings and retrieve details.

In [27]:
#  Complete the query by first finishing the CTE query  by adding the WHERE statement. Then complete the following SELECT AND FROM statements.

query = """

-- Create a common table expression (CTE) to select high-priced listings greater than 500
WITH high_price_listings AS (
    SELECT *
    FROM listings
    WHERE  price > 500 )

-- Select specific columns (id, name, neighbourhood_group, price) from high-priced listings
-- Limit the results to display only the top 5 listings

SELECT id, name, neighbourhood_group, price FROM  high_price_listings LIMIT 5;
"""

In [28]:


result = pd.read_sql_query(query, conn)
print("\n Using CTE for Clarity:")
print(result)


 Using CTE for Clarity:
       id                                   name neighbourhood_group  price
0   19601    perfect for a family or small group            Brooklyn    800
1   68974     Unique spacious loft on the Bowery           Manhattan    575
2   89427                   The Brooklyn Waverly            Brooklyn    650
3  103311      2 BR w/ Terrace @ Box House Hotel            Brooklyn    599
4  174966  Luxury 2Bed/2.5Bath Central Park View           Manhattan   2000


###  3.3: Using GROUP BY and HAVING
-  Count the number of listings in each neighbourhood_group with more than 1000 listings.

In [38]:
# :  the GROUP_BY and the HAVING COUNT statement to 1000 listings

query = """SELECT neighbourhood_group, COUNT(*) AS num_listings
FROM listings
GROUP BY neighbourhood_group
HAVING COUNT(*) >1000;
"""

In [39]:


result = pd.read_sql_query(query, conn)
print("\n - Using GROUP BY and HAVING:")
print(result)


 - Using GROUP BY and HAVING:
  neighbourhood_group  num_listings
0               Bronx          1091
1            Brooklyn         20104
2           Manhattan         21661
3              Queens          5666


###  3.4: Using [CASE Statements](https://www.w3schools.com/sql/sql_case.asp)
- Calculate the count of listings in each neighborhood group categorized into three price ranges: cheap, moderate, and expensive.


In [44]:
#  the third case switch statement for expensive_listings and insert GROUP BY statement

query = """

-- Select the neighborhood_group and use conditional aggregation to count listings in price ranges
SELECT neighbourhood_group,
       SUM(CASE WHEN price <= 100 THEN 1 ELSE 0 END) AS cheap_listings,
       SUM(CASE WHEN price > 100 AND price <= 200 THEN 1 ELSE 0 END) AS moderate_listings,
       SUM( CASE WHEN price > 200 THEN 1 ELSE 0 END) AS expensive_listings
FROM listings

-- Group the data by neighborhood_group to get counts for each group
GROUP BY neighbourhood_group;
"""

In [45]:


result = pd.read_sql_query(query, conn)
print("\nExample 5 - Using CASE Statements:")
print(result)


Example 5 - Using CASE Statements:
  neighbourhood_group  cheap_listings  moderate_listings  expensive_listings
0               Bronx             865                181                  45
1            Brooklyn           11771               6278                2055
2           Manhattan            6921               8803                5937
3              Queens            4101               1243                 322
4       Staten Island             270                 78                  25


From the above, it is found that :
expensive_listings, moderate_listings   are more in Manhattan and cheap_listings  in Brooklyn are more than in any other groups.

###  3.5: Using [Self-Join](https://www.w3schools.com/sql/sql_join_self.asp) for Data Comparison
- Compare the average price of each neighborhood's listings with the overall average price, highlighting neighborhoods with prices higher than the overall average.

In [58]:
query = """
-- Create a common table expression (CTE) to calculate the average prices for each neighborhood
WITH  avg_prices AS (
    SELECT neighbourhood, AVG(price) AS avg_price
    FROM listings
    GROUP BY neighbourhood
)

-- Select the neighborhood, its average price, and the overall average price
SELECT a.neighbourhood, a.avg_price, b.overall_avg_price
FROM avg_prices AS a

-- Self-join with a subquery to retrieve the overall average price
INNER JOIN ( SELECT AVG(price)  AS overall_avg_price FROM listings) AS b  -- No need to group in the subquery for overall average
ON 1=1  -- Join condition not needed for overall average comparison
WHERE a.avg_price > b.overall_avg_price
"""

result = pd.read_sql_query(query, conn)
print(" Using Self-Join for Data Comparison:")
print(result)

 Using Self-Join for Data Comparison:
         neighbourhood   avg_price  overall_avg_price
0              Arverne  171.779221         152.720687
1    Battery Park City  367.557143         152.720687
2              Bayside  157.948718         152.720687
3         Belle Harbor  171.500000         152.720687
4          Boerum Hill  176.135593         152.720687
5         Breezy Point  213.333333         152.720687
6     Brooklyn Heights  209.064935         152.720687
7      Carroll Gardens  175.914163         152.720687
8              Chelsea  249.738544         152.720687
9            Chinatown  161.497283         152.720687
10         City Island  173.000000         152.720687
11        Civic Center  191.942308         152.720687
12        Clinton Hill  181.893357         152.720687
13         Cobble Hill  211.929293         152.720687
14         Columbia St  162.952381         152.720687
15               DUMBO  196.305556         152.720687
16        East Village  186.083108         1

In [59]:


result = pd.read_sql_query(query, conn)
print(" Using Self-Join for Data Comparison:")
print(result)

 Using Self-Join for Data Comparison:
         neighbourhood   avg_price  overall_avg_price
0              Arverne  171.779221         152.720687
1    Battery Park City  367.557143         152.720687
2              Bayside  157.948718         152.720687
3         Belle Harbor  171.500000         152.720687
4          Boerum Hill  176.135593         152.720687
5         Breezy Point  213.333333         152.720687
6     Brooklyn Heights  209.064935         152.720687
7      Carroll Gardens  175.914163         152.720687
8              Chelsea  249.738544         152.720687
9            Chinatown  161.497283         152.720687
10         City Island  173.000000         152.720687
11        Civic Center  191.942308         152.720687
12        Clinton Hill  181.893357         152.720687
13         Cobble Hill  211.929293         152.720687
14         Columbia St  162.952381         152.720687
15               DUMBO  196.305556         152.720687
16        East Village  186.083108         1

# Section 4: Closing the Database Connection <a name="closing-connection"></a>

###  4.1: Close the Cursor and Database Connection
- It's good practice to close the cursor and the database connection when you're done working with the database to free up system resources and maintain proper connection management.


In [ ]:


# Make sure to include these lines of code at the end of your script to properly close the cursor and database connection.
cursor.close()
conn.close()

# Section 5: Summary of Basic SQL Commands <a name="summary-commands"></a>

 Below is a summary of the basic SQL commands used:

** 1.2: Connect to the Database**
- Established a connection to the SQLite database.

** 2.1: Load the Dataset**
- Loaded the dataset into a DataFrame.
- Imported the DataFrame into the database.

** 3.1: Using Subquery for Aggregation**
- SQL Commands: SELECT, FROM, WHERE, Subquery, MAX(), LIMIT
- Description: Find listings with a price greater than or equal to the maximum price of entire home/apartment listings.

** 3.2: Using CTE for Clarity**
- SQL Commands: WITH, Common Table Expression (CTE), SELECT, FROM, LIMIT
- Description: Create a Common Table Expression (CTE) for high-priced listings and retrieve details.

** 3.3: Using GROUP BY and HAVING**
- SQL Commands: SELECT, COUNT(), AS, FROM, GROUP BY, HAVING
- Description: Count the number of listings in each neighbourhood_group with more than 1000 listings.

** 3.4: Using CASE Statements**
- SQL Commands: SELECT, SUM(), CASE, WHEN, THEN, ELSE, END, AS, FROM, GROUP BY
- Description: Calculate the count of listings in each neighborhood group categorized into three price ranges: cheap, moderate, and expensive.

** 3.5: Using Self-Join for Data Comparison**
- SQL Commands: WITH, Common Table Expression (CTE), SELECT, AVG(), AS, INNER JOIN, ON, Self-Join, WHERE
- Description: Compare the average price of each neighborhood's listings with the overall average price, highlighting neighborhoods with prices higher than the overall average.

** 4.1: Close the Cursor and Database Connection**
- Closed the cursor and the database connection.

This summary provides an overview of the intermediate SQL commands used in this project on data analysis tasks.

**THE END**